In [79]:
import os, math, numpy as np
from copy import deepcopy

In [80]:
def get_travel_time(source: tuple, destination : tuple) -> float:
    return math.dist(source, destination) / 427
    # return ((source[1] - destination[1]) ** 2 + (source[0] - destination[0]) ** 2)**0.5 / 427

In [81]:
### Calculate travel time between two points: origin and destination
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

# Read in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [82]:
# Read instance information
def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    # locations.set_index('id',inplace=True)
    #
    # orders.set_index('order',inplace=True)
    # couriers.set_index('courier',inplace=True)
    # restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams

In [83]:
import pandas as pd
instance_dir = 'data/0o50t75s1p100'
orders,restaurants,couriers,instanceparams=read_instance_information(instance_dir)

In [84]:
instanceparams

,meters_per_minute,pickup service minutes,dropoff service minutes,target click-to-door,maximum click-to-door,pay per order,guaranteed pay per hour
0,427,4,4,40,90,10,15


# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon<br>
∆_(1); ∆_(2): to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [85]:
# Accorting to default values in the paper
f = 5
delta_u = 10
delta_1 = 10
delta_2 = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned

# Procedure to initialize a solution

**Function that return U(t,r)**<br>
U(t,r) is the set of upcoming orders at restaurant r

In [86]:
def create_set_U_tr(t,r):
    U_tr = orders[(orders['ready_time'] >= t) & (orders['ready_time'] <= t + delta_u) & (orders['restaurant'] == r)]
    return U_tr

**Courier Object**

In [87]:
class Order:
    def __init__(self, order_information : dict, restaurants : pd.DataFrame):

        self.order_id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')
        restaurants = restaurants.set_index('restaurant').to_dict()
        self.restaurant_location = (restaurants['x'][self.restaurant_id], restaurants['y'][self.restaurant_id])
        self.is_assigned = False

        self.order_fulfilment_time = get_travel_time(self.restaurant_location, self.destination)

    def __repr__(self):
        return f"order_id: {self.order_id}, restaurant_id: {self.restaurant_id}, destination: {self.destination}, placement_time : {self.placement_time}, ready_time : {self.ready_time} time_taken : {self.order_fulfilment_time}"

In [88]:
class Courier(object):
    def __init__(self, courier_information : dict, restaurant : dict):
        self.courier = courier_information.get('courier')
        self.x = courier_information.get('x')
        self.y = courier_information.get('y')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Update and derived along the way
        self.travel = []
        self.visited = []
        self.orders = []
        # self.next_available_time = # a dictionary containing restaurant as key and available time as value
        self.current_position = (self.x, self.y)
        self.current_restaurant = None
        self.distance_travelled = 0
        self.current_time = self.on_time
        self.restaurant = restaurant

    def __repr__(self):
        return "courier_id {} starting_coords {} on_time {} off_time {} current_position {} current_time {}".format(
            self.courier, (self.x, self.y), self.on_time, self.off_time, self.current_position, self.current_time
        )

    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple,
                              meters_per_minute : float) -> float:

        return get_travel_time(source, restaurant)  + get_travel_time(restaurant, destination)

   # check if the courier can take the order
    def can_assign(self, order : Order) -> bool:
        # if the ready time < courier's off time
        if order.ready_time < self.on_time:
            return False
        if order.ready_time > self.off_time:
            return False

    # assign an order to courier
    def assign_order(self, order: Order) -> bool:
        # update available time for the next assignment
        _, restaurant_x, restaurant_y = restaurants[restaurants['restaurant'] == order.restaurant].values[0]
        if self.can_assign(order):
            self.orders.append(order)
            self.current_time += self.calculate_travel_time(self.current_position, (restaurant_x, restaurant_y), orders.destination)

In [159]:
class DeliveryRouting:
    def __init__(self,
                 orders : pd.DataFrame,
                 restaurants : pd.DataFrame,
                 couriers : pd.DataFrame,
                 instanceparams: pd.DataFrame,
                 delta_u : int = 10,
                 target_bundle_size : int = 5):
        """
        :param t: ready time lower bound
        :param delta_u: tolerable
        :param target_bundle_size: Z_t
        """

        self.target_bundle_size = target_bundle_size
        self.delta_u = delta_u

        self.orders = [Order(order, restaurants) for order in orders.to_dict(orient = 'records')]
        self.orders = sorted(self.orders, key = lambda x: x.ready_time)
        self.restaurants = restaurants.set_index('restaurant').to_dict()
        self.couriers = [Courier(courier, self.restaurants) for courier in couriers.to_dict(orient = 'records')]

        self.unassigned_orders = deepcopy(self.orders)
        self.fulfilled_orders = []
        self.instance_params = instanceparams.to_dict(orient = 'records')[0]
        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.maximum_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = self.instance_params.values()

    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple = None ) -> float:

        total_time_taken = get_travel_time(source, restaurant)
        if destination is not None:
            total_time_taken += get_travel_time(restaurant, destination)
        return total_time_taken


    def initial_bundle(self, t : int):
        bundles = {}
        for order in self.orders:
            if order.ready_time <= t + self.delta_u:
                if order.restaurant_id not in bundles:
                    bundles[order.restaurant_id] = []
                bundles[order.restaurant_id].append(order)
        return bundles

    def remove_order(self, order_id):
        for order in self.unassigned_orders:
            if order.order_id == order_id:
                self.unassigned_orders.remove(order)

    def allocate_jobs(self, t : int) -> None:
        bundles = self.initial_bundle(t)
        for restaurant, bundle in bundles.items():
            for order in bundle:
                is_assigned = False
                if order.is_assigned == False:
                    min_time = 1e9
                    for courier in self.couriers:
                        if self.check_is_can_assign(courier, order, t):
                            x, y, z = self.check_is_can_assign(courier, order, t)
                            current_time = x+y+z
                            if current_time < min_time:
                                min_time = current_time
                                courier_assigned = courier
                                is_assigned = True
                                order.is_assigned = True
                                x1, y1, z1 = x,y,z

                if is_assigned:
                    # print(courier_assigned.courier, courier_assigned.current_time, min_time, x1, y1, z1)
                    courier_assigned.current_time = min_time + 4
                    courier_assigned.current_position = order.destination

                    self.remove_order(order.order_id)
                    self.fulfilled_orders.append(dict(order_id = order.order_id, courier_id =  courier_assigned.courier, fulfilled_time = min_time, restaurant_id = order.restaurant_id))


    def allocate_all_jobs(self):
        for t in range(0, 240, 10):
            self.allocate_jobs(t)



    def check_is_can_assign(self, courier, order, t):
        courier_start_moving = max(courier.current_time, order.placement_time)
        courier_start_leaving = max(order.ready_time, self.calculate_travel_time(courier.current_position, order.restaurant_location)) + 4
        courier_delivery_leg = self.calculate_travel_time(order.restaurant_location, order.destination) + 4

        if order.order_id not in [order.order_id for order in self.unassigned_orders]:
            return False

        if courier.on_time > t:
            return False

        if order.ready_time > courier.off_time:
            return False

        if courier_start_leaving > courier.off_time:
            return False

        if courier_start_moving + courier_start_moving + courier_delivery_leg - 4 > order.placement_time + 90:
            return False

        return courier_start_moving, courier_start_leaving, courier_delivery_leg

In [160]:
dr = DeliveryRouting(orders, restaurants, couriers, instanceparams)
dr.allocate_all_jobs()

In [162]:
dr.fulfilled_orders

[{'order_id': 'o146',
  'courier_id': 'c1',
  'fulfilled_time': 60.100510131831214,
  'restaurant_id': 'r54'},
 {'order_id': 'o89',
  'courier_id': 'c2',
  'fulfilled_time': 69.9661704350347,
  'restaurant_id': 'r50'},
 {'order_id': 'o121',
  'courier_id': 'c1',
  'fulfilled_time': 154.1840248956962,
  'restaurant_id': 'r13'},
 {'order_id': 'o163',
  'courier_id': 'c2',
  'fulfilled_time': 184.50070809638424,
  'restaurant_id': 'r39'}]

In [163]:
dr.unassigned_orders

[order_id: o240, restaurant_id: r67, destination: (9386, 5834), placement_time : 29, ready_time : 46 time_taken : 4.912795248347009,
 order_id: o227, restaurant_id: r39, destination: (9334, 6307), placement_time : 33, ready_time : 48 time_taken : 10.721196118073529,
 order_id: o159, restaurant_id: r67, destination: (10213, 5982), placement_time : 36, ready_time : 56 time_taken : 5.534378047101177,
 order_id: o184, restaurant_id: r31, destination: (6070, 4473), placement_time : 98, ready_time : 105 time_taken : 6.029227952956611,
 order_id: o174, restaurant_id: r21, destination: (5442, 4861), placement_time : 98, ready_time : 108 time_taken : 3.997251961497653,
 order_id: o110, restaurant_id: r55, destination: (10145, 5621), placement_time : 108, ready_time : 111 time_taken : 7.114333573238854,
 order_id: o226, restaurant_id: r1, destination: (7601, 5739), placement_time : 102, ready_time : 112 time_taken : 4.876668919728399,
 order_id: o234, restaurant_id: r90, destination: (10441, 292